In [1]:
# importation des bibliothèques
import requests
import pandas as pd
import sqlite3, sqlalchemy
from tqdm.notebook import trange, tqdm
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect, Float

In [2]:
# création de la base de données 
engine = create_engine('sqlite:///test.db', echo=False)
meta = MetaData()

In [3]:
import pandas as pd

# Create sample data using a dictionary
ratings_data = {
    'title_id': ['tt0111161', 'tt0068646', 'tt0468569', 'tt0071562', 'tt0050083',
                 'tt0108052', 'tt0167260', 'tt0110912', 'tt0060196', 'tt0137523'],
    'rating': [9.3, 9.2, 9.0, 9.0, 8.9, 8.9, 8.9, 8.8, 8.8, 8.8],
    'votes': [2800000, 1900000, 2700000, 1300000, 780000,
              1400000, 1900000, 2100000, 760000, 2200000]
}

# Create DataFrame
ratings_sample = pd.DataFrame(ratings_data)

# Display the table
print(ratings_sample)

    title_id  rating    votes
0  tt0111161     9.3  2800000
1  tt0068646     9.2  1900000
2  tt0468569     9.0  2700000
3  tt0071562     9.0  1300000
4  tt0050083     8.9   780000
5  tt0108052     8.9  1400000
6  tt0167260     8.9  1900000
7  tt0110912     8.8  2100000
8  tt0060196     8.8   760000
9  tt0137523     8.8  2200000


In [8]:
# Basic read
df = pd.DataFrame(ratings_sample)
# print(df.head())  # Affiche les premières lignes du DataFrame
# Écrire dans la base de données

# transformation des données du dataframe en liste de tuple
values = df.to_records(index=False).tolist()


# création de la table ratings
ratings = Table(
    'ratings', meta, 
    Column('title_id', String, primary_key=True), 
    Column('rating', Float), 
    Column('votes', String),
    extend_existing=True)

meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=ratings.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

# drop des data ratings
df = []
values = []

In [9]:
# affichage du noms des tables dans la base de données newIMDB.db
inspector = inspect(engine)
inspector.get_table_names()

['ratings']

In [10]:
# affichage des noms de variables pour toutes les tables de la base newIMDB.db
for table_name in inspector.get_table_names():
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'])

Column: title_id
Column: rating
Column: votes


In [11]:
# test de requete SQL

engineIMDB = create_engine('sqlite:///test.db')
connIMDB = engineIMDB.connect()

result = connIMDB.execute("SELECT title_id,rating,votes" 
                          " FROM ratings"  
                          " WHERE votes > 5000" 
                          " ORDER BY rating" 
                          " DESC, ratings.votes" 
                          " DESC LIMIT 5")
result.fetchall()

[('tt0050083', 8.9, '780000'), ('tt0060196', 8.8, '760000')]

In [12]:
import sqlite3
import re

SQLITE_DB = "test.db"
OUTPUT_SQL = "test_dump.sql"

TYPE_MAPPING = {
    "INTEGER": "INT",
    "REAL": "DOUBLE",
    "TEXT": "TEXT",
    "BLOB": "BLOB",
    "NUMERIC": "DECIMAL(10,5)"
}

def map_sqlite_type(sqlite_type: str) -> str:
    if not sqlite_type:
        return "TEXT"
    sqlite_type = sqlite_type.upper()
    for k, v in TYPE_MAPPING.items():
        if k in sqlite_type:
            return v
    return "TEXT"

def escape(value):
    if value is None:
        return "NULL"
    if isinstance(value, (int, float)):
        return str(value)
    return "'" + str(value).replace("'", "''") + "'"

conn = sqlite3.connect(SQLITE_DB)
cursor = conn.cursor()

with open(OUTPUT_SQL, "w", encoding="utf-8") as f:
    f.write("SET foreign_key_checks = 0;\n\n")

    # List tables
    cursor.execute("""
        SELECT name FROM sqlite_master
        WHERE type='table' AND name NOT LIKE 'sqlite_%'
    """)
    tables = [row[0] for row in cursor.fetchall()]

    for table in tables:
        # Get schema
        cursor.execute(f"PRAGMA table_info({table})")
        columns = cursor.fetchall()

        col_defs = []
        for cid, name, col_type, notnull, default, pk in columns:
            mysql_type = map_sqlite_type(col_type)
            col_def = f"`{name}` {mysql_type}"
            if pk:
                col_def += " PRIMARY KEY AUTO_INCREMENT"
            elif notnull:
                col_def += " NOT NULL"
            col_defs.append(col_def)

        f.write(f"DROP TABLE IF EXISTS `{table}`;\n")
        f.write(f"CREATE TABLE `{table}` (\n  ")
        f.write(",\n  ".join(col_defs))
        f.write("\n) ENGINE=InnoDB;\n\n")

        # Dump data
        cursor.execute(f"SELECT * FROM {table}")
        col_names = [desc[0] for desc in cursor.description]

        for row in cursor:
            values = ", ".join(escape(v) for v in row)
            f.write(
                f"INSERT INTO `{table}` ({', '.join(f'`{c}`' for c in col_names)}) "
                f"VALUES ({values});\n"
            )

        f.write("\n")

    f.write("SET foreign_key_checks = 1;\n")

conn.close()

print("MySQL-compatible SQL dump created.")


MySQL-compatible SQL dump created.
